In [1]:
# CS6510 HW 1 Code Skeleton
# Please use this outline to implement your decision tree. You can add any code around this.
import csv
import math
import numpy as np
import time
# Enter You Name Here
myname = "Sharath" # or "Doe-Jane-"

In [2]:
# Calculate the initial entropy for a set of rows by converting it into numpy array and passing it to entropy()
# to calculate entropy for the set of rows
def calculateInitialEntropy(training_set):
    training_set=np.asarray(training_set).astype("float")
    initial_entropy = entropy(training_set)
    return initial_entropy

In [3]:
#Calculate the information gain based on the parentEntropy and the newly calculated entropy
def calculateIG(parentEntropy, entropy):
    return (parentEntropy-entropy)

In [4]:
#Alternate function to calculate entropy using the split count of rows based on the two labels
'''def entropyForm(class0count, class1count):
    if(class0count>0 and class1count>0):
        probclass0= float(class0count/(class0count+class1count))
        probclass1=1-probclass0
    else:
        probclass0=0
        probclass1=0
        return 0
    entropyCal= probclass0*(-1)*math.log(probclass0)+ probclass1*(-1)*math.log(probclass1)
    return entropyCal'''

'def entropyForm(class0count, class1count):\n    if(class0count>0 and class1count>0):\n        probclass0= float(class0count/(class0count+class1count))\n        probclass1=1-probclass0\n    else:\n        probclass0=0\n        probclass1=0\n        return 0\n    entropyCal= probclass0*(-1)*math.log(probclass0)+ probclass1*(-1)*math.log(probclass1)\n    return entropyCal'

In [5]:
#Alternate function to get the entropy calculation for the column for only 2 labels
'''
def getEntropy(training_set, column_index, value):
    classg=[row for row in training_set if float(row[column_index])>=value]
    classl=[row for row in training_set if float(row[column_index])< value]
    count_classg= len(classg)
    count_classl=len(classl)
    
    if(count_classg==0 and count_classl==0):
        prob_classg=0
        prob_classl=0
    else:
        prob_classg= float(count_classg/(count_classg+ count_classl))
        prob_classl= 1-prob_classg
        
    classg0= [row for row in classg if float(row[-1])==0]
    count_classg0=len(classg0)
    classl0= [row for row in classl if float(row[-1])==0]
    count_classl0=len(classl0)
    count_classg1= len(classg)-count_classg0
    count_classl1=len(classl)-count_classl0
    
    entropyg= entropyForm(count_classg0, count_classg1)
    entropyl= entropyForm(count_classl0, count_classl1)
    
    entropyVal= prob_classg*entropyg+ prob_classl* entropyl
    
    return entropyVal'''

'\ndef getEntropy(training_set, column_index, value):\n    classg=[row for row in training_set if float(row[column_index])>=value]\n    classl=[row for row in training_set if float(row[column_index])< value]\n    count_classg= len(classg)\n    count_classl=len(classl)\n    \n    if(count_classg==0 and count_classl==0):\n        prob_classg=0\n        prob_classl=0\n    else:\n        prob_classg= float(count_classg/(count_classg+ count_classl))\n        prob_classl= 1-prob_classg\n        \n    classg0= [row for row in classg if float(row[-1])==0]\n    count_classg0=len(classg0)\n    classl0= [row for row in classl if float(row[-1])==0]\n    count_classl0=len(classl0)\n    count_classg1= len(classg)-count_classg0\n    count_classl1=len(classl)-count_classl0\n    \n    entropyg= entropyForm(count_classg0, count_classg1)\n    entropyl= entropyForm(count_classl0, count_classl1)\n    \n    entropyVal= prob_classg*entropyg+ prob_classl* entropyl\n    \n    return entropyVal'

In [6]:
# get the entropy for a set of rows using the formula : - summation(p*log2p) where p is the probab of rows that belong to one label
def entropy(training_set):
    if(training_set.shape[0]==0):
        return 0
    training_set1=training_set[:,-1]
    unique_elements, counts_elements = np.unique(training_set1, return_counts=True)# get the count and type of label in the rows
    dic=dict(zip(unique_elements,counts_elements))# create a dictionary with the label and count 
    p = []
    for label in dic:
        p.append(dic[label]/float(training_set1.shape[0])) # get the probability of every label into an array so that it is easy for numpy calculation
    return -np.sum(np.multiply(p,np.log2(p)))

In [7]:
#get the entropy for a column by multipling the entropy formula by the probability of split rows
def getEntropy(training_set, column_index, value):
    training_set_left = [row for row in training_set if float(row[column_index]) >= value]
    training_set_right = [row for row in training_set if float(row[column_index])< value]
    training_set_left = np.asarray(training_set_left,dtype='float')
    training_set_right = np.asarray(training_set_right,dtype='float')
    p =float((training_set_left.shape[0])/(training_set_left.shape[0]+training_set_right.shape[0]))
    return p*entropy(training_set_left)+(1-p)*entropy(training_set_right)
    
    

In [8]:
# find the best split for a set of rows by comparing the entropy of all the columns and choosing the one with
#highest information gain
def findSplit(training_set):
    local_entropy=0
    local_column=None
    best_entropy=1
    best_column=0
    col_mean_final=0
    bestIg=0
    parentEntropy=calculateInitialEntropy(training_set)
    for col in range(0,len(training_set[0])-1):
        col_data= [float(row[col]) for row in training_set]
        col_mean=float(np.sum(col_data)/len(col_data))
        true_rows, false_rows= partition(training_set,col,col_mean)
        if(len(true_rows) == 0 or len(false_rows)==0):
            continue
        local_entropy= getEntropy(training_set,col,col_mean)
        localIg= calculateIG(parentEntropy,local_entropy)
        if localIg>= bestIg:  # store the column only when the information gain of a column is maximum
            bestIg=localIg
            best_entropy=local_entropy
            best_column = col
            col_mean_final=col_mean
    return  bestIg,best_column,col_mean_final

In [9]:
#Split the rows into true and false based on the way in which they satisfy the condition for the column 
def partition(training_set, column_index, value):
    true_rows= [row for row in training_set if float(row[column_index]) >= value]
    false_rows = [row for row in training_set if float(row[column_index])< value]
    return true_rows, false_rows

In [10]:
# get the class of data in case it cant be split further i.e all the data in this particular class belong to
# the same class. So we just take the first label and set it as the output label for the decision tree node
def getClass(training_set, column_index,value):
    true_label= None
    false_label=None
    true_rows= [row for row in training_set if float(row[column_index]) >= value]
    false_rows = [row for row in training_set if float(row[column_index])< value]
    if len(true_rows)>0:
        true_label = true_rows[0][-1]
    if len(false_rows)>0:
        false_label= false_rows[0][-1]
    if false_label=="1":
        false_label="class1"
    if true_label=="0":
        true_label="class0"
    if false_label=="0":
        false_label="class0"
    if true_label=="1":
        true_label="class1"
    if len(true_rows)> len(false_rows):
        return true_label
    else:
        return false_label

In [11]:
class DecisionTree():
    tree = {}

    def learn(self, training_set,index):
        #implement this function
        value=0
        true_index=0
        false_index=0
        entropy,column_name,value = findSplit(training_set)
        if entropy==0: # Base case when entropy is 0 aka impurity is zero
            return getClass(training_set, column_name, value) # we get the class and return it
        true_rows, false_rows = partition(training_set,column_name,value)

        true_index= self.learn(true_rows, 2*index)
        false_index= self.learn(false_rows,2*index+1)
        self.tree[index]=[column_name,value,{"true_rows":true_index,"false_rows":false_index}]
        return index

    def classify(self, test_instance):
        #print(self.tree)
        result = 0 # baseline: always classifies as 0
        index=1
        while 1:
            value= self.tree[index]
            column_num= value[0]
            values=value[1]
            if float(test_instance[column_num])>=float(values):
                result= value[2]["true_rows"]
            else:
                #print(value[2])
                result=value[2]["false_rows"]
            if result=="class1" or result=="class0":
                break
            else:
                index=result
        if result=="class1":
            result="1"
        else:
            result="0"
        return result


In [12]:
def run_decision_tree():

    # Load data set
    with open("assign1-q1-data.csv") as f:
        next(f, None)
        data = [tuple(line) for line in csv.reader(f, delimiter=",")]
    print("Number of records: %d" % len(data))

    # Split training/test sets
    # You need to modify the following code for cross validation.
    K = 10
    training_set = [x for i, x in enumerate(data) if i % K !=7]
    test_set = [x for i, x in enumerate(data) if i % K == 7]
    print(type(training_set))
    columns_set={}
    tree = DecisionTree()
    tree.learn( training_set,1)
    results = []
#     print(training_set)
#     print(test_set)
    
    for instance in test_set:
        result = tree.classify( instance[:-1] ) # prints all values except the last one
        results.append( result == instance[-1])
    accuracy = float(results.count(True))/float(len(results))
    print("Final accuracy: %.4f" % accuracy)       
    f = open(myname+"result.txt", "w")
    f.write("accuracy: %.4f" % accuracy)
    f.close()

In [13]:
if __name__ == "__main__":
    run_decision_tree()

Number of records: 4898
<class 'list'>
Final accuracy: 0.8265
